# 1. Introduction

In this report, we derive the closed-form solution for the price of a zero-coupon bond under the [Vasicek (1977)] interest rate model. 
This process involves working with a **Stochastic Differential Equation (SDE)**, and applying two different techniques to solve it:

1. **Stochastic Calculus Approach** (using **Ito’s Lemma**)  
2. **Deterministic PDE Approach** (using the **Feynman-Kac Theorem**)

Our goal is to provide clear, step-by-step explanations that can be understood even by readers with no background in mathematics or finance.

Additionally, we will explain how the theoretical derivations are directly linked to the Python code we implemented, which simulates interest rates, performs parameter estimation using Maximum Likelihood, and applies Kalman Filtering techniques.

Understanding these derivations is crucial for interpreting the dynamics of bond pricing models and for implementing them correctly in practice.

# 2. The Vasicek Model – Overview

The **Vasicek model** is a classical short-rate model that describes how the short-term interest rate evolves over time. It is widely used in bond pricing and term structure modeling.

The model assumes that the short rate $r_t$ follows the following **Stochastic Differential Equation (SDE)** under the **risk-neutral measure** $\mathbb{Q}$:

$$
dr_t = \kappa_Q (\theta_Q - r_t)dt + \sigma dW_t^{\mathbb{Q}}
$$



###  Parameters:

- $r_t$: short-term interest rate at time $t$  
- $\kappa_Q$: speed of mean reversion  
- $\theta_Q$: long-term mean of the interest rate  
- $\sigma$: volatility of the short rate  
- $W_t^{\mathbb{Q}}$: standard Brownian motion under the risk-neutral measure  



###  Intuition:

- The process is **mean-reverting**, meaning that $r_t$ tends to return to its long-run average $\theta_Q$.
- The speed of this return is governed by $\kappa_Q$.
- The randomness comes from the Brownian motion $dW_t^{\mathbb{Q}}$, scaled by the volatility $\sigma$.



###  Objective:

Our goal is to derive the **price of a zero-coupon bond** under this interest rate model. Specifically, we want to compute the present value of 1 unit of money received at maturity $T$:

$$
P(t, T) = \mathbb{E}_t^{\mathbb{Q}} \left[ \exp \left( -\int_t^T r_s ds \right) \right]
$$

In the next sections, we will derive this expression using:

1. **Method 1**: Stochastic Calculus (Ito’s Lemma)  
2. **Method 2**: Partial Differential Equation (Feynman-Kac Theorem)


# 3. Method 1: Extended Derivation Using Ito's Lemma (with Full Integral Steps)

We derive the closed-form bond pricing formula under the Vasicek model using Ito’s Lemma, with a focus on clarity and full mathematical steps — especially for the integral solution of \( A(\tau) \).



###  Step 0: Objective

We want to find the bond price under the Vasicek model:

$$
P(t, T) = \mathbb{E}_t^{\mathbb{Q}}\left[ \exp\left( -\int_t^T r_s ds \right) \right]
$$

This is the expected discounted value of 1 unit of money received at time \( T \), discounted continuously with short rate \( r_s \).



###  Step 1: Guess the Solution Form

Assume the bond price has the following exponential-affine form:

$$
P(t, T) = \exp\left( -A(\tau) - B(\tau) r_t \right)
$$

Where:
- $$ \tau = T - t $$
- \( A(\tau), B(\tau) \) are functions of time to maturity, to be determined



###  Step 2: Model SDE

Under the risk-neutral measure:

$$
dr_t = \kappa_Q (\theta_Q - r_t) dt + \sigma dW_t^{\mathbb{Q}}
$$



###  Step 3: Apply Ito's Lemma

Let:

$$
P(t, r_t) = e^{-A(\tau) - B(\tau) r_t}
$$

We apply Ito’s Lemma. Needed derivatives:

- $$ \frac{\partial P}{\partial t} = ( -A'(\tau) + B'(\tau) r_t ) P $$
- $$ \frac{\partial P}{\partial r_t} = -B(\tau) P $$
- $$ \frac{\partial^2 P}{\partial r_t^2} = B(\tau)^2 P $$

Then Ito's Lemma gives:

$$
\frac{dP}{P} = \left[ -A'(\tau) - B'(\tau) r_t - \kappa_Q (\theta_Q - r_t) B(\tau) + \frac{1}{2} \sigma^2 B(\tau)^2 \right] dt - \sigma B(\tau) dW_t
$$



###  Step 4: Martingale Condition

Risk-neutral pricing requires:

$$
\text{drift of } \frac{dP}{P} = r_t
$$

So equating the drift term:

$$
r_t = -A'(\tau) - B'(\tau) r_t - \kappa_Q (\theta_Q - r_t) B(\tau) + \frac{1}{2} \sigma^2 B(\tau)^2
$$



###  Step 5: Match Coefficients

Group the terms by powers of \( r_t \):

1. **\( r_t \) terms:**

$$
-B'(\tau) + \kappa_Q B(\tau) = 1
$$

→ First-order linear ODE for B(τ)

2. **Constant terms:**

$$
-A'(\tau) - \kappa_Q \theta_Q B(\tau) + \frac{1}{2} \sigma^2 B(\tau)^2 = 0
$$

→ Defines ODE for A(τ)



###  Step 6: Solve for B(τ)

The ODE is:

$$
B'(\tau) = \kappa_Q B(\tau) - 1
$$

Solve using integrating factor method:

Let integrating factor: $$ e^{-\kappa_Q \tau} $$

Multiply both sides:

$$
e^{-\kappa_Q \tau} B'(\tau) - \kappa_Q e^{-\kappa_Q \tau} B(\tau) = -e^{-\kappa_Q \tau}
$$

Left-hand side is derivative:

$$
\frac{d}{d\tau} \left[ e^{-\kappa_Q \tau} B(\tau) \right] = -e^{-\kappa_Q \tau}
$$

Integrate both sides:

$$
e^{-\kappa_Q \tau} B(\tau) = \int_0^\tau -e^{-\kappa_Q s} ds + C
$$

Compute the integral:

$$
\int_0^\tau -e^{-\kappa_Q s} ds = \left[ \frac{-1}{\kappa_Q} e^{-\kappa_Q s} \right]_0^\tau = \frac{-1}{\kappa_Q} (e^{-\kappa_Q \tau} - 1)
$$

So:

$$
e^{-\kappa_Q \tau} B(\tau) = \frac{1}{\kappa_Q}(1 - e^{-\kappa_Q \tau})
$$

Multiply both sides by $$ e^{\kappa_Q \tau} $$ :

$$
B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q}
$$



###  Step 7: Solve for A(τ)

From earlier:

$$
A'(\tau) = -\kappa_Q \theta_Q B(\tau) + \frac{1}{2} \sigma^2 B(\tau)^2
$$

Plug in expression for B(τ):

Let:

$$
B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q}
$$

Then:

$$
A'(\tau) = -\kappa_Q \theta_Q \cdot \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} + \frac{1}{2} \sigma^2 \cdot \left( \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} \right)^2
$$

Simplify:

$$
A'(\tau) = -\theta_Q (1 - e^{-\kappa_Q \tau}) + \frac{\sigma^2}{2 \kappa_Q^2} (1 - e^{-\kappa_Q \tau})^2
$$

Integrate both sides with respect to A(τ):

$$
A(\tau) = \int_0^\tau \left[ -\theta_Q (1 - e^{-\kappa_Q s}) + \frac{\sigma^2}{2 \kappa_Q^2} (1 - e^{-\kappa_Q s})^2 \right] ds
$$

Break into two parts:

**Part 1:**

$$
\int_0^\tau -\theta_Q (1 - e^{-\kappa_Q s}) ds = -\theta_Q \left[ s + \frac{1}{\kappa_Q} e^{-\kappa_Q s} \right]_0^\tau = -\theta_Q \left( \tau - \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} \right)
$$

**Part 2:**

Let’s denote:

$$
I = \int_0^\tau (1 - e^{-\kappa_Q s})^2 ds
$$

We expand the square:

$$
I = \int_0^\tau \left( 1 - 2e^{-\kappa_Q s} + e^{-2\kappa_Q s} \right) ds
= \left[ s + \frac{2}{\kappa_Q} e^{-\kappa_Q s} - \frac{1}{2\kappa_Q} e^{-2\kappa_Q s} \right]_0^\tau
$$

Evaluate:

$$
I = \tau - \frac{2}{\kappa_Q}(1 - e^{-\kappa_Q \tau}) + \frac{1}{2\kappa_Q}(1 - e^{-2\kappa_Q \tau})
$$

Thus:

$$
\displaystyle
A(\tau) = -\theta_Q \left( \tau - \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} \right) + \frac{\sigma^2}{2 \kappa_Q^2} \left( \tau - \frac{2}{\kappa_Q}(1 - e^{-\kappa_Q \tau}) + \frac{1}{2\kappa_Q}(1 - e^{-2\kappa_Q \tau}) \right)
$$



###  Final Result

The bond price is:

$$
P(t, T) = \exp\left( -A(\tau) - B(\tau) r_t \right)
$$

With:

$$
B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q}
$$

and

$$
A(\tau) = -\theta_Q \left( \tau - \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} \right)
+ \frac{\sigma^2}{2 \kappa_Q^2} \left( \tau - \frac{2}{\kappa_Q}(1 - e^{-\kappa_Q \tau}) + \frac{1}{2\kappa_Q}(1 - e^{-2\kappa_Q \tau}) \right)
$$


# 4. Method 2: Derivation Using the Feynman-Kac Theorem (Fully Expanded)

We now derive the Vasicek bond price using the **Feynman-Kac theorem**, showing all steps in full mathematical detail.



###  Objective

We want to compute:

$$
P(t, T) = \mathbb{E}_t^{\mathbb{Q}}\left[ e^{-\int_t^T r_s ds} \right]
$$

Let’s define:

$$
V(t, r) := P(t, T)
$$

We aim to derive a PDE that this function satisfies.



###  Step 1: PDE from Feynman-Kac

Under the Vasicek model under the risk-neutral measure:

$$
dr_t = \kappa_Q (\theta_Q - r_t) dt + \sigma dW_t^{\mathbb{Q}}
$$

The Feynman-Kac theorem implies that the function \( V(t, r) \) satisfies the following PDE:

$$
\frac{\partial V}{\partial t} + \kappa_Q (\theta_Q - r) \frac{\partial V}{\partial r} + \frac{1}{2} \sigma^2 \frac{\partial^2 V}{\partial r^2} - r V = 0
$$

with the terminal condition:

$$
V(T, r) = 1
$$



###  Step 2: Guess the Functional Form

Assume:

$$
V(t, r) = e^{-A(\tau) - B(\tau) r}
$$

where $$ \tau = T - t $$, and $$ A(\tau) $$, $$ B(\tau) $$ are functions to be determined.



###  Step 3: Compute Derivatives

Let’s calculate the derivatives of $$ V(t, r) = e^{-A(\tau) - B(\tau) r} $$

- Time derivative:

$$
\frac{\partial V}{\partial t} = -\frac{dA}{d\tau} \cdot \frac{d\tau}{dt} \cdot V - \frac{dB}{d\tau} \cdot r \cdot \frac{d\tau}{dt} \cdot V = (A'(\tau) + B'(\tau) r) V
$$

- First derivative w.r.t. \( r \):

$$
\frac{\partial V}{\partial r} = -B(\tau) V
$$

- Second derivative:

$$
\frac{\partial^2 V}{\partial r^2} = B(\tau)^2 V
$$



###  Step 4: Plug into the PDE

Substitute into:

$$
\frac{\partial V}{\partial t}
+ \kappa_Q (\theta_Q - r) \frac{\partial V}{\partial r}
+ \frac{1}{2} \sigma^2 \frac{\partial^2 V}{\partial r^2}
- r V = 0
$$

Each term becomes:

- $$ \frac{\partial V}{\partial t} = (A'(\tau) + B'(\tau) r) V $$
- $$ \kappa_Q (\theta_Q - r)(-B(\tau) V) = -\kappa_Q \theta_Q B(\tau) V + \kappa_Q r B(\tau) V $$
- $$ \frac{1}{2} \sigma^2 B(\tau)^2 V $$
- $$ -r V $$

Putting them all together:

$$
(A'(\tau) + B'(\tau) r) V
- \kappa_Q \theta_Q B(\tau) V
+ \kappa_Q r B(\tau) V
+ \frac{1}{2} \sigma^2 B(\tau)^2 V
- r V = 0
$$

Divide both sides by \( V \) (since \( V > 0 \)):

$$
A'(\tau) + B'(\tau) r - \kappa_Q \theta_Q B(\tau) + \kappa_Q r B(\tau) + \frac{1}{2} \sigma^2 B(\tau)^2 - r = 0
$$



###  Step 5: Group Terms

Group by powers of \( r \):

- Coefficient of \( r \):

$$
B'(\tau) + \kappa_Q B(\tau) = 1
$$

- Constant term:

$$
A'(\tau) = \kappa_Q \theta_Q B(\tau) - \frac{1}{2} \sigma^2 B(\tau)^2
$$

These are the same ODEs we obtained using Ito’s Lemma.



###  Step 6: Solve for B(τ)

We solve:

$$
B'(\tau) = 1 - \kappa_Q B(\tau)
$$

Use integrating factor $\mu(\tau) = e^{\kappa_Q \tau}$


$$
\frac{d}{d\tau} \left[ e^{\kappa_Q \tau} B(\tau) \right] = e^{\kappa_Q \tau}
$$

Integrate both sides:

$$
e^{\kappa_Q \tau} B(\tau) = \int_0^\tau e^{\kappa_Q s} ds
= \left[ \frac{1}{\kappa_Q} e^{\kappa_Q s} \right]_0^\tau
= \frac{1}{\kappa_Q}(e^{\kappa_Q \tau} - 1)
$$

Multiply by $$ e^{-\kappa_Q \tau} $$:

$$
B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q}
$$



###  Step 7: Solve for A(τ)

We have:

$$
A'(\tau) = \kappa_Q \theta_Q B(\tau) - \frac{1}{2} \sigma^2 B(\tau)^2
$$

Insert $$ B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} $$ and integrate as shown in detail in the previous section.



###  Final Formula

$$
P(t, T) = e^{-A(\tau) - B(\tau) r_t}
$$

with:

$$
B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q}
$$

$$
A(\tau) = -\theta_Q \left( \tau - \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} \right)
+ \frac{\sigma^2}{2 \kappa_Q^2} \left( \tau - \frac{2}{\kappa_Q}(1 - e^{-\kappa_Q \tau}) + \frac{1}{2\kappa_Q}(1 - e^{-2\kappa_Q \tau}) \right)
$$


# 5. Connecting Mathematical Derivation to Python Code

In our Python implementation, we use the Vasicek model to simulate interest rates, estimate parameters using Maximum Likelihood Estimation (MLE), and apply the Kalman Filter to estimate latent short rates.

Let us now explain how the mathematical derivations relate to the actual code.



###  Model Assumption

The Vasicek model under the **risk-neutral measure** is:

$$
dr_t = \kappa_Q (\theta_Q - r_t) dt + \sigma dW_t^{\mathbb{Q}}
$$

The solution to this stochastic differential equation (SDE) has the following **closed-form**:

$$
r_{t+\Delta t} = \theta_Q + (r_t - \theta_Q) e^{-\kappa_Q \Delta t} + \sigma \sqrt{\frac{1 - e^{-2\kappa_Q \Delta t}}{2\kappa_Q}} \cdot \eta_t
$$

where $$ \eta_t \sim \mathcal{N}(0, 1) $$.

This formula is used in the code to simulate the short rate path $$ r_t $$ under the risk-neutral dynamics.



###  Code Output: Bond Prices

After simulating $$ r_t $$, we compute theoretical **zero-coupon bond prices** using the derived formula:

$$
P(t, T) = e^{-A(\tau) - B(\tau) r_t}
$$

with:

- $$ B(\tau) = \frac{1 - e^{-\kappa_Q \tau}}{\kappa_Q} $$
- $$ A(\tau) = \text{(derived in detail earlier)} $$

In the notebook, this is applied for multiple maturities (e.g. 1M, 6M, 1Y, 3Y, 5Y, 10Y) to build a full **yield curve**.



###  Estimation Process in Code

We perform **Maximum Likelihood Estimation (MLE)** using observed bond yields to recover:

- $$ \kappa_P, \theta_P, \alpha_P $$: under the physical measure (P)
- $$ \kappa_Q, \theta_Q, \alpha_Q $$: under the risk-neutral measure (Q)
- $$ \sigma $$: volatility of short rate

The estimated parameters are then used to **rebuild the Vasicek model**, recompute the yield curve, and apply the **Kalman Filter** for latent state estimation.



###  Kalman Filter and the Vasicek Model

Kalman Filter is used to estimate the **latent short rate $$ r_t $$** over time.

The state-space representation in the code corresponds to:

- **State equation**: dynamics of $$ r_t $$ from the Vasicek model
- **Observation equation**: yield curve generated from bond prices using the derived formulas

Hence, every layer of the code has a direct theoretical justification based on the derivations shown earlier.


# 6. Economic and Financial Interpretation

Now that we have derived the Vasicek bond pricing formula using both **Ito’s Lemma** and the **Feynman-Kac theorem**, and implemented it in Python, we conclude with the economic interpretation of the results.



###  What Does the Vasicek Model Tell Us?

The Vasicek model describes how the **short-term interest rate** (often denoted as $$ r_t $$) evolves over time. It captures three important economic properties:

1. **Mean Reversion**:  
   The term $$ \kappa_Q (\theta_Q - r_t) $$ ensures that the interest rate tends to return to a long-run mean level $$ \theta_Q $$. This reflects the idea that interest rates don’t drift away forever — they are pulled back by monetary policy, inflation control, and macroeconomic forces.

2. **Stochastic Fluctuations**:  
   The volatility term $$ \sigma dW_t $$ represents random shocks due to market news, central bank actions, or economic data releases. These shocks cause short-term fluctuations.

3. **No Arbitrage Pricing**:  
   The bond price formula:

   $$
   P(t, T) = e^{-A(\tau) - B(\tau) r_t}
   $$

   ensures that bond prices are consistent with the **no arbitrage principle** under the risk-neutral measure.



###  How Do A(τ) and B(τ) Affect Bond Prices?

- **B(τ)** determines the sensitivity of bond price to the current short rate $$ r_t $$.  
  As maturity increases, B(τ) increases and then flattens — this matches the intuition that long-term bonds are more sensitive to interest rates.

- **A(τ)** adjusts the level of the bond price for risk premiums and volatility.



###  Yield Curve Implications

The model allows us to construct a **yield curve**, i.e., the term structure of interest rates. Depending on the parameters:

- If $$ \theta_Q > r_t $$: upward-sloping yield curve  
- If $$ \theta_Q < r_t $$: downward-sloping yield curve  
- The steepness depends on $$ \kappa_Q $$ and $$ \sigma $$

This is crucial for:
- Valuing interest rate derivatives
- Pricing fixed income instruments
- Making investment decisions in bond markets



###  Real-World Relevance

While the Vasicek model is relatively simple, it forms the foundation for more advanced models like CIR, Hull-White, and affine term structure models.

It helps analysts, risk managers, and quants understand:

- How interest rates behave
- How bond prices react to changing rates
- How to hedge portfolios against rate risk



###  Summary

The Vasicek model provides an elegant and tractable way to model interest rates and price bonds. Through our derivations and implementation, we now understand both the **theoretical mechanics** and the **practical consequences** of the model.
